### Problem Set 7: Multitape Turing Machine

csc427: Theory of Automata and Complexity. 
<br>
university of miami
<br>
spring 2021.
<br>
Burton Rosenberg.
<br>
<br>
created: April 15, 2021
<br>last update: April 19, 2021


---

### Student name:

---

### Overview

Besides introducing multi-tape turing machines, the project makes the connect between language recognition and computation. As an example: we could as for an accept or reject outcome for the three numbers n1, n2, n3, where the accept outcome is only when n3 = n1 + n2. 

However, we will produce n3 from n1 and n2, making it more like a standard use of a computer. 

The multi-tape machine will also allow us to introduce many interesting models. Non-deterministic computing can be modeled as a deterministic computation with a read only tape of random numbers. Oracle machines can be introduced where the TM writes a question on a tape and pauses while the question is replaced with an answer.



### TuringMachineMT Thin

Clone or copy csc427_code_ps7.py and place in the same directory as this notebook. Then we can have a less cluttered Notebook by importing the code.

<code>
&delta; ( qi, a1, a2 ) = ( qj, b1, b2, X, Y)    
</code>

where X and Y can be r, l, n, R, L or N. And here that would be rendered,

<code>
state: qi
    a1 a2 b1 b2 X Y qj
</code>


In [1]:
from csc427_code_ps7 import TuringMachineMT, MachineParserMT

### Exercise A: copy

The standard definition of the multitape Turing Machine starts with the input written into tape 0 and all other tapes are blank. All heads are on the leftmost cell. 

A common operation is to copy from the 0th tape to the other tape, a delimited section of input. Here we demonstrate a program which copies all of the 0th tape onto the 1st tape.


In [2]:
tmmt_00 = """# copy t1 to t2

start: q0
accept: A
reject: R
tapes: 2

state: q0
    : : : : n n R

"""

tmmt_test = [
    "0",
    "01",
    "101"
]


def test_tmmt_00(tm_description, test_cases,verbose='none'):
    tm = TuringMachineMT()
    MachineParserMT.turing(tm,tm_description)
 
    print("\n*** TEST RUNS")
    correct = 0
    for s in test_cases:
        # assume complexity is some quadratic
        res = tm.compute_tm(s,step_limit=10*(len(s)+5)**2,verbose='none')
        if res==True or res==False:
            tapes = tm.get_tapes()
            for i in range(len(tapes)):  
                print(f'{i}:\t{tapes[i]}')
            print()
            if tapes[0]==tapes[1]:
                correct +=1
        else:
            print(f'ERROR on input {s}: {TuringMacine[tm.result]}') 
    print(f"*** {correct} out of {len(test_cases)} correct\n")


test_tmmt_00(tmmt_00,tmmt_test,verbose='explain')  



*** TEST RUNS
0:	0
1:	_

0:	01
1:	_

0:	101
1:	_

*** 0 out of 3 correct



### Exercise B: Move To Work Tape


In [3]:
tmmt_01 = """# move to work tape
# precondition- tape 0: &a&b tape 1: __
# postcondition - tape 0: &a tape 1: &b

start: q0
accept: A
reject: R
tapes: 2

state: q0
    : : : : n n R
    
"""

tmmt_test = [
    '&0&1',
    '&0&101',
    '&101&0',
    '&111&11111'
]

In [4]:

def test_tmmt_01(tm_description, test_cases,verbose='none'):

    tm = TuringMachineMT()
    MachineParserMT.turing(tm,tm_description)
 
    print("\n*** TEST RUNS")

    correct = 0
    for test in test_cases:
        # assume complexity is some quadratic
        res = tm.compute_tm(test,step_limit=10*(len(test[0])+5)**2,verbose=verbose)
        if res==True or res==False:
            tapes = tm.get_tapes()
            if test == tapes[0]+tapes[1]:
                correct += 1
                print(f'correct: {test} -> {tapes[0]}, {tapes[1]}')
            else:
                print(f'incorrect: {test} -> {tapes[0]}, {tapes[1]}')
        else:
            print(f'ERROR on input {test_pair}: {TuringMachine[tm.result]}')

    print(f"*** {correct} out of {len(test_cases)} correct\n")

# TuringMachineMT.help()


test_tmmt_01(tmmt_01,tmmt_test,verbose='none')  



*** TEST RUNS
incorrect: &0&1 -> &0&1, _
incorrect: &0&101 -> &0&101, _
incorrect: &101&0 -> &101&0, _
incorrect: &111&11111 -> &111&11111, _
*** 0 out of 4 correct



### Exercise C: Adding in binary

On a two tape machine, write a program that adds binary numbers <em>Note well!</em> For the purposes of this project we will write our numbers from left to right, with the Least Significant Bit the leftmost bit in the number. This makes things a little bit simpler.

The input will be written on the 0th tape as,

<pre>
    &dashv; &amp; n1 &amp; n2
</pre>
where n1 and n2 are binary numbers, reversed written, and the TM should leave this tape with the answer, n3 = n1 + n2, 

<pre>
     &dashv; &amp; n3
</pre>

My suggestion is to copy n2 to the second tape, erasing it with the separating ampersand. Another hint is to then zero pad so both n1 and n2 are the same length. Then add the tape 1 number into the tape 0 number, so it is left in place.

A number and a number with additional leading zeros are equivalent. They can be given that way as input or delivered as output, without consequence for correctness.


In [5]:
tmmt_02 = """# addition
# &n1&n2 -> n1+n2

start: q0
accept: A
reject: R
tapes: 2

state: q0
   : : : : n n R
    
"""

tmmt_test = [
    ('&0&0','&0'),
    ('&1&1','&01'),
    ('&1&01','&11'),
    ('&01&1','&11'),
    ('&011&01','&0001')
]


In [6]:

def test_tmmt_02(tm_description, test_cases,verbose='none'):
    
    def zero_pad(n1,n2):
        if len(n1)==len(n2):
            return (n1,n2)
        if len(n1)<len(n2):
            n1, n2 = n2, n1
        return (n1, n2+'0'*(len(n1)-len(n2)))

    tm = TuringMachineMT()
    MachineParserMT.turing(tm,tm_description)
 
    print("\n*** TEST RUNS")

    correct = 0
    for test_pair in test_cases:
        # assume complexity is some quadratic
        res = tm.compute_tm(test_pair[0],step_limit=10*(len(test_pair[0])+5)**2,verbose=verbose)
        if res==True or res==False:
            tapes = tm.get_tapes()
            for i in range(len(tapes)):  
                print(f'{i}:\t{tapes[i]}')
            n1, n2 = zero_pad(tapes[0],test_pair[1])
            if n1 == n2:
                correct += 1
                print(f'correct: {test_pair[0]} should be {n2} and is {n1}')
            else:
                print(f'incorrect: {test_pair[0]} should be {n2} and is {n1}')
        else:
            print(f'ERROR on input {test_pair}: {TuringMachine[tm.result]}')

    print(f"*** {correct} out of {len(test_cases)} correct\n")

# TuringMachineMT.help()


test_tmmt_02(tmmt_02,tmmt_test,verbose='none')  



*** TEST RUNS
0:	&0&0
1:	_
incorrect: &0&0 should be &000 and is &0&0
0:	&1&1
1:	_
incorrect: &1&1 should be &010 and is &1&1
0:	&1&01
1:	_
incorrect: &1&01 should be &1100 and is &1&01
0:	&01&1
1:	_
incorrect: &01&1 should be &1100 and is &01&1
0:	&011&01
1:	_
incorrect: &011&01 should be &000100 and is &011&01
*** 0 out of 5 correct



### Exercise D: Subtracting in Binary

Much of the situation as the same as above, except leave on the 0th tape <code> n1 - n2</code>.

While neither n1 nor n2 will be negative, the result can be. If the result is negative replace the &amp; with a %.  For instance, 

<code>
    &dashv; &amp; 11 &amp; 101   &xrarr; &dashv; % 01
</code>

You might want to use 2's complement, because it is easy to 2's complement a number and you will reuse your addition code of the previous exercise. You will have to sufficiently pad the the numbers so that an overflow means the result is negative. That should trigger code to adjust the representation of the result.

There is a short help-explanation about 2's completement below.

In [7]:
tmmt_03 = """# subtraction
# &n1&n2 -> &n3 or %n3, where n3 = n1-n2 and the % signifies the number is negative

start: q0
accept: A
reject: R
tapes: 2

state: q0
   : : : : n n R

"""

tmmt_test = [
    ('&0&0','&0'),
    ('&1&1','&0'),
    ('&1&01','%1'),
    ('&01&1','&1'),
    ('&011&01','&001')
]



#### Two's complement

The secret of 2's complement to represent negative numbers is to know that there are not negative or positive numbers at all. The number system is actually the integers modulo N = 2<sup>n</sup> for n bits, and half of these we call negative and half we call positive.

In the integers, given an integer, the other integers are either "before" or "after" this integer. Those after we count up to. Those before we count down to. However modulo N we can count forward to any other integer, as well could backwards to any other integer.

In the integers modulo N, count back one from 0 gives N-1. For N=16, this means 15. And as,

<code>
     15 + 1 = 16 = 0 modulo 16
</code>

then 15 acts like -1. We can decide whether to call it 15 or -1, as suits our purposes.

Given an integer i modulo N, think of it as a binary number in n bits. It is important we think of the n bits, not just the value of i. Consider the complement of i, &sim; i. The sum of i + &sim; i is obviously whatever value is the n-bit number of all ones.

An example:

<code>
    i = 6 =  0101<sub>2</sub>
    &sim;i = &sim;0101<sub>2</sub> = 1010<sub>2</sub> = 9
    i + &sim;i  = 0101<sub>2</sub> + 1010<sub>2</sub> = 1111<sub>2</sub> = 15
</code>

Adding one, ignoring the overflow, the result of a number plus its complement is 0. 

<code>
    i + &sim;i + 1 = 0     modulo N
    -i = (&sim;i + 1)    modulo N
</code>

We can make this look like positive and negative numbers by dividing out the N numbers into N/2 positive numbers, that should include 1, 2, 3 ... and some others; and N/2 negative numbers, which should include -1, -2, -3 ... and some others. We know from two's complement that -1 = 1111<sub>2</sub> in the case of 4 bits, so we conclude that if the most significant bit is 1, let us believe the number is negative; otherwise it is positive. 

We can just leave a negative number be, for instance, there is nothing wrong with 1110<sub>2</sub>. However for this assignment I would like you exchange the version of the negative number with the leading bit 1 with a negative indicator (the % replacing the &amp;) followed by the negative of the negative. 

There is still an issue, however. In this system,

<code>
    7 + 7 = 14 = -2 modulo 16
</code>

and that doesn't seem right. Two positives should not add up to a negative. There was an overflow that placed the resulting large positive into the set of integers we call negative.


The answer is to use enought bits that things do not overflow. Since the largest number with a 0 in the leading bit is N/2-1 (for n = 2, that is the number 0111<sub>2</sub> or 7) we must not add numbers larger than N/4. So we select n to be the number of bits required to represent the larger number we will attempt to add, and put on 2 more bits of precision. Now you can do the two's complement and we will not have arithmetic overflow.

For instance, now 7 + 7 is ok,

<code>
    7 = 00111<sub>2</sub>,
    00111<sub>2</sub> = 00111<sub>2</sub>, = 01110<sub>2</sub> = 14
</code>

And so is -7 + -7, 

<code>
    -7 = &sim;00111<sub>2</sub> + 1 = 11001<sub>2</sub>,
    11001<sub>2</sub> + 11001<sub>2</sub> = 10010<sub>2</sub>
    since &sim;10010<sub>2</sub> + 1 = 01110<sub>2</sub> = 14,
    10010<sub>2</sub> = -14
</code>


In [8]:

def test_tmmt_03(tm_description, test_cases,verbose='none'):
    test_tmmt_02(tmmt_03,tmmt_test,verbose='none') 

test_tmmt_03(tmmt_03,tmmt_test,verbose='explain')  



*** TEST RUNS
0:	&0&0
1:	_
incorrect: &0&0 should be &000 and is &0&0
0:	&1&1
1:	_
incorrect: &1&1 should be &000 and is &1&1
0:	&1&01
1:	_
incorrect: &1&01 should be %1000 and is &1&01
0:	&01&1
1:	_
incorrect: &01&1 should be &1000 and is &01&1
0:	&011&01
1:	_
incorrect: &011&01 should be &001000 and is &011&01
*** 0 out of 5 correct



### Extra Credit: Verifying Graph Paths

The Turing Machine takes a Graph and a list of vertices, and accepts if the list of vertices is a path in the graph. Else it rejects.

Vertices are named by 0-1 strings. The graphs are directed and are specified as an ampersand separated sequence of edges. Each edge is two vertex names separated by a comma. The sequence of edges is terminated by double ampersands. Befoe the first edge is a dollar sign, to help find the left of the tape, and collowing is the list of vertices as a comma separate sequence of vertex names.

Here is an example that is accepted.

<code>
000 ---&gt; 001 ---&gt; 010
          |        |
          V        V
         011 ---&gt; 100
is    
    &dashv;&dollar;000,001&amp;001,010&amp;001,011&amp;011,100&amp;010,110&amp;&amp;001,011,100
</code>

As a suggestion &mdash; copy the trailing list of vertices to tape 1, erasing it from tape 0, 

<code>
    &dashv;&dollar;000,001&amp;001,010&amp;001,011&amp;011,100&amp;010,110
    &dashv;&dollar;001,011,100
</code>

Then scan tape 1 looking for exact matches of pairs of vertex names. Example, first can tape 0 for the exact match for <code>001,011</code>. If no match, reject; else move on to <code>011,100</code>

In [9]:
tmmt_04 = """# paths in directed graphs
# graph descripion & & a path

start: q0
accept: A
reject: R
tapes: 2

state: q0
    & & _ _ n n R
    

"""

In [10]:
def test_tmmt_04(tm_description, test_cases,verbose='none'):
    pass

test_tmmt_04(tmmt_04,tmmt_test,verbose='none')  
